# Iterative SQD (iSQD)

In [1]:
from copy import deepcopy
from functools import partial
import h5py
import numpy as np

import pyscf
from pyscf import ao2mo
from pyscf.tools.fcidump import from_scf, read

import openfermion as of

from qiskit import QuantumRegister, QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.qasm2 import dumps
from qiskit_aer import AerSimulator
import qiskit_aer.noise as noise
import qiskit_ibm_runtime
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_addon_sqd.fermion import diagonalize_fermionic_hamiltonian, solve_sci_batch, SCIResult

import ffsim

from adaptvqe.utils import hamiltonian_from_fcidump
from adaptvqe.hamiltonians import FermionicHamiltonian
from adaptvqe.pools import DVG_CEO
from adaptvqe.algorithms.adapt_vqe import TensorNetAdapt


In [2]:
ibm_computer: str = "ibm_fez"

service = qiskit_ibm_runtime.QiskitRuntimeService(channel="local")
computer = service.backend()
sampler = Sampler(computer)

# prob_1 = 0.001
# prob_2 = 0.01

# error_1 = noise.depolarizing_error(prob_1, 1)
# error_2 = noise.depolarizing_error(prob_2, 2)

# noise_model = noise.NoiseModel()
# noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3'])
# noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

# computer = AerSimulator(noise_model=noise_model)
# sampler = Sampler(computer)

/home/ryan/prof/work/wellcome/tensoradapt/envtensoradapt-new/lib/python3.10/site-packages/qiskit_ibm_runtime/fake_provider/backends/nighthawk/fake_nighthawk.py:76: UserWarning: Properties of fake_nighthawk are not intended to represent typical nighthawk error values.
  warnings.warn(


In [3]:
# mol = pyscf.gto.Mole()
# geom = of.chem.geometry_from_pubchem("water")
# basis = "sto-3g"
# symmetry = "C1"
# multiplicity = 1
# charge = 0
# mol.build(
#     atom=geom,
#     basis=basis,
#     symmetry=symmetry,
# )
# mf = pyscf.scf.RHF(mol)
# mf.kernel()
# lih_fname = "data/water.fcidump"
# from_scf(mf, lih_fname)

In [4]:
lih_fname = "data/atp_0_fcidump/atp_0_be1_f0"

In [6]:
h_fcidump, norb, nelec = hamiltonian_from_fcidump(lih_fname)
h = FermionicHamiltonian(h_fcidump, "atp", nelec, diag_mode="quimb", max_mpo_bond=10, max_mps_bond=20)
nq = h.n
print(f"Hamiltonian has {nq} qubits.")

Hamiltonian has 24 qubits.


In [7]:
# Since we have alpha and beta e-'s, an n-qubit Hamiltonian as n/2 spatial orbitals.
assert h.n % 2 == 0
num_orbitals = h.n // 2

In [ ]:
# TODO Get the exact energy from PySCF.
exact_energy = h.ground_energy
print(f"Got exact energy {exact_energy}.")

In [8]:
# Get one- and two-body integrals.
fci_read = read(lih_fname)
h1 = fci_read["H1"]
h2_packed = fci_read["H2"]
h2 = ao2mo.restore(1, h2_packed, num_orbitals)  # (norb,norb,norb,norb)
n_electrons = fci_read["NELEC"]
ecore = fci_read["ECORE"]
spin = 0 # TODO How would I know from the FCIDUMP alone?
num_elec_a = (n_electrons + spin) // 2
num_elec_b = (n_electrons - spin) // 2
nelec = (num_elec_a, num_elec_b)

Parsing data/atp_0_fcidump/atp_0_be1_f0


In [9]:
import time

In [10]:
start = time.monotonic()
pool = DVG_CEO(n=h.n)
stop = time.monotonic()
print(f"Finished building pool in {stop - start} seconds. Has {len(pool.operators)} operators.")

Finished building pool in 15479.54300818406 seconds. Has 26466 operators.


In [11]:
(stop - start) / 60 / 60

4.299873057828906

In [18]:
max_mpo_bond = 8
adapt_mps_bond = 16
my_adapt = TensorNetAdapt(
    pool=pool,
    custom_hamiltonian=h,
    max_adapt_iter=1,
    recycle_hessian=True,
    tetris=True,
    verbose=True,
    threshold=0.1,
    max_mpo_bond=max_mpo_bond,
    max_mps_bond=adapt_mps_bond,
)
my_adapt.initialize()


tensor-net-adapt prepared with the following settings:
> Pool: DVG_CEO
> Custom Hamiltonian: atp
> Orbital Optimization: False
> Selection method: gradient
> Convergence criterion: total_g_norm
> Recycling Hessian: True
> Tetris: True (progressive optimization: False)
> Convergence threshold (gradient norm):  0.1
> Maximum number of iterations:  1
> candidates per iteration:  1

Initial energy: -52.25129918638107


In [19]:
circuits = []
adapt_energies = []
for i in range(32):
    print(f"On iteration {i}.")
    my_adapt.run_iteration()
    data = my_adapt.data
    ansatz_circuit = pool.get_circuit(my_adapt.indices, my_adapt.coefficients)
    print("coefficients:", my_adapt.coefficients)
    print("indices:", my_adapt.indices)
    # Prepare the HF reference state, then add the Ansatz circuit.
    q = QuantumRegister(2 * num_orbitals)
    circuit = QuantumCircuit(q)
    circuit.append(ffsim.qiskit.PrepareHartreeFockJW(num_orbitals, nelec), q)
    circuit = circuit.compose(ansatz_circuit)
    circuit.measure_all()
    circuits.append(circuit)
    adapt_energies.append(my_adapt.energy)

On iteration 0.

*** ADAPT-VQE Iteration 1 ***

Creating list of up to 26466 operators ordered by gradient magnitude...

Non-Zero Gradients (tolerance E-8):
Operator 2: -0.006348190615281907
Operator 3: -0.01050280567959059
Operator 4: -0.0065284241631455775
Operator 5: -0.041311879928597715
Operator 6: -0.04235274043107596
Operator 7: 0.21450985686596535
Operator 8: -0.06537034835246897
Operator 9: 0.10159645298158779
Operator 10: 0.005835697942210891
Operator 13: 0.0028088980304145526
Operator 14: 0.0004316095987053772
Operator 15: 0.001434791001222316
Operator 16: 0.002660309138119617
Operator 17: 0.0037583821262357517
Operator 18: -0.014188885349129985
Operator 19: 0.005808367019764072
Operator 20: -0.0014476623625623948
Operator 21: -0.003663530847061473
Operator 23: -0.032945731189716755
Operator 24: 0.019517332221192788
Operator 25: -0.015638136017501546
Operator 26: 0.02860974615033516
Operator 27: 0.0164275484916668
Operator 28: -0.2875203556958496
Operator 29: -0.019229821014

/home/ryan/prof/work/wellcome/tensoradapt/envtensoradapt-new/lib/python3.10/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: barrier
  warnings.warn(
/home/ryan/prof/work/wellcome/tensoradapt/ben/ceo-adapt-vqe/adaptvqe/algorithms/adapt_vqe.py:2492: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  opt_result = minimize_bfgs(
/home/ryan/prof/work/wellcome/tensoradapt/ben/ceo-adapt-vqe/adaptvqe/algorithms/adapt_vqe.py:2507: UserWarning: Optimizer did not succeed. Message: Desired error not necessarily achieved due to precision loss.
  warn(f"Optimizer did not succeed. Message: {opt_result.message}")
/home/ryan/prof/work/wellcome/tensoradapt/envtensoradapt-new/lib/python3.10/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: barrier
  warnings.warn(


         Current function value: -52.302396
         Iterations: 5
         Function evaluations: 40
         Gradient evaluations: 30

Current energy: -52.30239568984797
(change of -0.05109650346690131)
Current ansatz: [np.int64(56), np.int64(28), np.int64(49), np.int64(8), np.int64(37), np.int64(19)]
coefficients: [np.float64(-0.17622482276506204), np.float64(0.054581803581400516), np.float64(0.08832165825365432), np.float64(9.266897808928998e-05), np.float64(-0.004734744905006992), np.float64(-0.002549109360232861)]
indices: [56 28 49  8 37 19]
On iteration 1.

*** ADAPT-VQE Iteration 2 ***

Creating list of up to 26466 operators ordered by gradient magnitude...

Non-Zero Gradients (tolerance E-8):
Operator 0: -0.010752333605873514
Operator 1: -0.008901759587852047
Operator 2: -0.005893500533741451
Operator 3: -0.009982116606543627
Operator 4: -0.006007660623025254
Operator 5: -0.03839370814867625
Operator 6: -0.03897721118188717
Operator 7: 0.19684958814493353
Operator 8: -0.064143

/home/ryan/prof/work/wellcome/tensoradapt/envtensoradapt-new/lib/python3.10/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: barrier
  warnings.warn(
/home/ryan/prof/work/wellcome/tensoradapt/ben/ceo-adapt-vqe/adaptvqe/algorithms/adapt_vqe.py:2492: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  opt_result = minimize_bfgs(
/home/ryan/prof/work/wellcome/tensoradapt/ben/ceo-adapt-vqe/adaptvqe/algorithms/adapt_vqe.py:2507: UserWarning: Optimizer did not succeed. Message: Desired error not necessarily achieved due to precision loss.
  warn(f"Optimizer did not succeed. Message: {opt_result.message}")
/home/ryan/prof/work/wellcome/tensoradapt/envtensoradapt-new/lib/python3.10/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: barrier
  warnings.warn(


         Current function value: -52.325214
         Iterations: 8
         Function evaluations: 63
         Gradient evaluations: 51

Current energy: -52.32521397527319
(change of -0.022818285425223905)
Current ansatz: [np.int64(56), np.int64(28), np.int64(49), np.int64(8), np.int64(37), np.int64(19), np.int64(7), np.int64(51), np.int64(48), np.int64(30), np.int64(19), np.int64(108), np.int64(37)]
coefficients: [np.float64(-0.18976160570438622), np.float64(0.06607707129876873), np.float64(0.08672636148654232), np.float64(-0.0001062540611984942), np.float64(-0.0046145637313124346), np.float64(0.00015237879938704603), np.float64(-0.0075535823858188965), np.float64(0.2611879670724583), np.float64(-0.005166254475060947), np.float64(-0.04856012693133454), np.float64(0.001924433325799571), np.float64(-4.265201958069103e-16), np.float64(-0.00034286758310483266)]
indices: [ 56  28  49   8  37  19   7  51  48  30  19 108  37]
On iteration 2.

*** ADAPT-VQE Iteration 3 ***

Creating list of up

/home/ryan/prof/work/wellcome/tensoradapt/envtensoradapt-new/lib/python3.10/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: barrier
  warnings.warn(
/home/ryan/prof/work/wellcome/tensoradapt/ben/ceo-adapt-vqe/adaptvqe/algorithms/adapt_vqe.py:2492: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  opt_result = minimize_bfgs(
/home/ryan/prof/work/wellcome/tensoradapt/ben/ceo-adapt-vqe/adaptvqe/algorithms/adapt_vqe.py:2507: UserWarning: Optimizer did not succeed. Message: Desired error not necessarily achieved due to precision loss.
  warn(f"Optimizer did not succeed. Message: {opt_result.message}")
/home/ryan/prof/work/wellcome/tensoradapt/envtensoradapt-new/lib/python3.10/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: barrier
  warnings.warn(


         Current function value: -52.335312
         Iterations: 12
         Function evaluations: 54
         Gradient evaluations: 42

Current energy: -52.335311741286
(change of -0.01009776601280521)
Current ansatz: [np.int64(56), np.int64(28), np.int64(49), np.int64(8), np.int64(37), np.int64(19), np.int64(7), np.int64(51), np.int64(48), np.int64(30), np.int64(19), np.int64(108), np.int64(37), np.int64(47), np.int64(54), np.int64(9), np.int64(29), np.int64(19), np.int64(75), np.int64(37), np.int64(86)]
coefficients: [np.float64(-0.18911309685315636), np.float64(0.05646503907628875), np.float64(0.07681590507448126), np.float64(0.0007342688167257347), np.float64(-0.00178128519784441), np.float64(-0.006116967112083027), np.float64(-0.004443034531425512), np.float64(0.266899864282937), np.float64(-0.0030561378483964993), np.float64(-0.04830857133042795), np.float64(0.0030914791942993334), np.float64(-6.588206053563753e-16), np.float64(-0.0005334600731405825), np.float64(0.0701437586704

/home/ryan/prof/work/wellcome/tensoradapt/envtensoradapt-new/lib/python3.10/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: barrier
  warnings.warn(
/home/ryan/prof/work/wellcome/tensoradapt/ben/ceo-adapt-vqe/adaptvqe/algorithms/adapt_vqe.py:2492: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  opt_result = minimize_bfgs(
/home/ryan/prof/work/wellcome/tensoradapt/ben/ceo-adapt-vqe/adaptvqe/algorithms/adapt_vqe.py:2507: UserWarning: Optimizer did not succeed. Message: Desired error not necessarily achieved due to precision loss.
  warn(f"Optimizer did not succeed. Message: {opt_result.message}")
/home/ryan/prof/work/wellcome/tensoradapt/envtensoradapt-new/lib/python3.10/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: barrier
  warnings.warn(


         Current function value: -52.335309
         Iterations: 0
         Function evaluations: 60
         Gradient evaluations: 48

Current energy: -52.33530850013577
(change of 3.2411502317586383e-06)
Current ansatz: [np.int64(56), np.int64(28), np.int64(49), np.int64(8), np.int64(37), np.int64(19), np.int64(7), np.int64(51), np.int64(48), np.int64(30), np.int64(19), np.int64(108), np.int64(37), np.int64(47), np.int64(54), np.int64(9), np.int64(29), np.int64(19), np.int64(75), np.int64(37), np.int64(86), np.int64(4312), np.int64(4313), np.int64(45), np.int64(8), np.int64(19), np.int64(75), np.int64(37), np.int64(66), np.int64(84)]


IndexError: list index out of range

In [15]:
circuits[0].draw(fold=-1)

┌───────────────────┐                                                                         ░                                                                          ░                                                                          ░                                                                            ░                                                                            ░                                                                            ░  ░ ┌─┐                                                                     
   q0_0: ┤0                  ├─────────────────────────────────────────────────────────────────────────░──────────────────────────────────────────────────────────────────────────░──────────────────────────────────────────────────────────────────────────░────────────────────────────────────────────────────────────────────────────░────────────────────────────────────────────────────────────────────────────░────────────────────────────────────────────────────────────────────────────░──░─┤M├─────────────────────────────────────────────────────────────────────
         │                   │                                                                         ░                                                                          ░                                                                          ░                                                                            ░                                                                            ░                                                                            ░  ░ └╥┘┌─┐                                                                  
   q0_1: ┤1                  ├─────────────────────────────────────────────────────────────────────────░──────────────────────────────────────────────────────────────────────────░──────────────────────────────────────────────────────────────────────────░────────────────────────────────────────────────────────────────────────────░────────────────────────────────────────────────────────────────────────────░────────────────────────────────────────────────────────────────────────────░──░──╫─┤M├──────────────────────────────────────────────────────────────────
         │                   │                                                                         ░                                                                          ░                                                                          ░                                                                            ░                                                                            ░                                                                            ░  ░  ║ └╥┘┌─┐                                                               
   q0_2: ┤2                  ├─────────────────────────────────────────────────────────────────────────░──────────────────────────────────────────────────────────────────────────░──────────────────────────────────────────────────────────────────────────░────────────────────────────────────────────────────────────────────────────░────────────────────────────────────────────────────────────────────────────░────────────────────────────────────────────────────────────────────────────░──░──╫──╫─┤M├───────────────────────────────────────────────────────────────
         │                   │                                                                         ░                                                                          ░ ┌─────────┐┌─────────┐     ┌──────────────┐     ┌──────────┐┌──────────┐ ░                                                                            ░                                                                            ░                                                                            ░  ░  ║  ║ └╥┘┌─┐                                                            
   q0_3: ┤3                  ├─────────────────────────────────────────────

In [ ]:
spin_a_layout = list(range(0, 12))
spin_b_layout = [12, 13, 14, 15, 19, 35, 34, 33, 32, 31, 30, 29]
initial_layout = spin_a_layout + spin_b_layout

In [ ]:
pass_manager = generate_preset_pass_manager(
    optimization_level=3, backend=computer, initial_layout=initial_layout[:nq]
)
pass_manager.pre_init = ffsim.qiskit.PRE_INIT

to_run = []
for circuit in circuits:
    # TODO: Remove barriers.
    compiled = pass_manager.run(circuit)
    print(compiled.count_ops())
    to_run.append(compiled)

In [ ]:
nshots: int = 10

In [ ]:
job = sampler.run(to_run, shots=nshots)  # TODO: Set nshots for each circuit.

In [ ]:
bit_arrays = [result.data.meas for result in job.result()]
counts_list = [bit_array.get_counts() for bit_array in bit_arrays]

In [ ]:
energies = []
errors = []
for bit_array in bit_arrays:
    # SQD options
    energy_tol = 1e-5
    occupancies_tol = 1e-6
    max_iterations = 20
    rng = np.random.default_rng(1)

    # Eigenstate solver options
    num_batches = 2
    samples_per_batch = 1000
    symmetrize_spin = True
    carryover_threshold = 1e-4
    max_cycle = 200

    sci_solver = partial(solve_sci_batch, spin_sq=0.0, max_cycle=max_cycle)

    # List to capture intermediate results
    result_history = []
    def callback(results: list[SCIResult]):
        result_history.append(results)
        # iteration = len(result_history)
        # print(f"Iteration {iteration}")
        # for i, result in enumerate(results):
        #     print(f"\tSubsample {i}")
        #     print(f"\t\tEnergy: {result.energy + ecore}") # + nuclear_repulsion_energy}")
        #     print(f"\t\tSubspace dimension: {np.prod(result.sci_state.amplitudes.shape)}")


    result = diagonalize_fermionic_hamiltonian(
        h1,
        h2,
        bit_array,
        samples_per_batch=samples_per_batch,
        norb=num_orbitals,
        nelec=nelec,
        num_batches=num_batches,
        energy_tol=energy_tol,
        occupancies_tol=occupancies_tol,
        max_iterations=max_iterations,
        sci_solver=sci_solver,
        symmetrize_spin=symmetrize_spin,
        carryover_threshold=carryover_threshold,
        callback=callback,
        seed=rng,
    )

    min_e = [
        min(result, key=lambda res: res.energy).energy + ecore # + nuclear_repulsion_energy
        for result in result_history
    ]
    sqd_energy = min(min_e)
    err = abs(min(min_e) - exact_energy)
    print("SQD Energy:", sqd_energy)
    print("Absolute energy error:", err)
    energies.append(sqd_energy)
    errors.append(err)

In [ ]:
qasm_strings = []
for circuit in circuits:
    isa_circuit = pass_manager.run(circuit)
    qasm_str = dumps(isa_circuit)
    qasm_strings.append(qasm_str)



In [ ]:
molec_name = "LiH"
f = h5py.File(f"data/{molec_name}.hdf5", "w")
f.create_dataset("energies", data=np.array(energies))
f.create_dataset("circuit_qasm", data=qasm_strings)